# CPS Microdata Processing

**Processing CPS microdata using Python and Polars**

## Purpose

This notebook processes raw CPS data:
- Loads raw CPS data (4.4M observations)
- Applies all sample selection filters
- Creates demographic cells
- Aggregates to skilled/unskilled
- Outputs processed labor series

## Why Polars?

- **Speed**: 5-10x faster than pandas on large data
- **Memory**: More efficient with 4.4M rows
- **API**: Similar to pandas, easy to learn

## Outputs

This notebook produces:
- `labor_totl.csv` - Aggregate time series
- `labor_by_group.csv` - Demographic cell data
- Sample selection statistics

---

## 1. Setup and Configuration

In [6]:
# Install polars if needed (uncomment)
# !pip install polars

import polars as pl
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

print("Polars version:", pl.__version__)

# Paths
ROOT = Path.cwd().parent
DATA_RAW = ROOT / 'data' / 'raw'
DATA_PROC = ROOT / 'data' / 'proc'

print(f"\nData paths:")
print(f"  Raw: {DATA_RAW}")
print(f"  Processed: {DATA_PROC}")
print(f"\nRaw CPS file exists: {(DATA_RAW / 'cps_00022.csv').exists()}")

Polars version: 1.33.0

Data paths:
  Raw: /Users/mitchv34/Work/industry_skill_premium/data/raw
  Processed: /Users/mitchv34/Work/industry_skill_premium/data/proc

Raw CPS file exists: True


## 2. Load Raw CPS Data

Loading 4.4M observations from IPUMS CPS extract.

In [10]:
# Load raw CPS with Polars (ensure HFLAG is numeric)
print("Loading raw CPS data...")
print(f"File: {cps_file}")
print(f"Size: {cps_file.stat().st_size / (1024**3):.2f} GB")

# Read with explicit dtype for HFLAG (nullable Int64)
df = pl.read_csv(cps_file, dtypes={'HFLAG': pl.Int64})

print(f"\nLoaded: {df.shape[0]:,} rows × {df.shape[1]} columns")
print(f"Memory: {df.estimated_size('mb'):.1f} MB")

# Confirm HFLAG is numeric and show its distribution
print("\nSchema:")
print(df.schema)
print("\nHFLAG distribution:")
print(df.group_by('HFLAG').count().sort('HFLAG'))

print(f"\nFirst few rows:")
print(df.head())

Loading raw CPS data...
File: /Users/mitchv34/Work/industry_skill_premium/data/raw/cps_00022.csv
Size: 0.31 GB

Loaded: 4,358,291 rows × 21 columns
Memory: 555.6 MB

Schema:
Schema({'YEAR': Int64, 'HFLAG': Int64, 'ASECWTH': Float64, 'CPI99': Float64, 'ASECWT': Float64, 'AGE': Int64, 'SEX': Int64, 'RACE': Int64, 'EMPSTAT': Int64, 'OCC2010': String, 'OCC1990': String, 'IND1990': String, 'AHRSWORKT': Int64, 'EDUC': Int64, 'HIGRADE': Int64, 'EDUC99': String, 'CLASSWLY': Int64, 'WKSWORK1': String, 'WKSWORK2': Int64, 'UHRSWORKLY': String, 'INCWAGE': Int64})

HFLAG distribution:
shape: (3, 2)
┌───────┬─────────┐
│ HFLAG ┆ count   │
│ ---   ┆ ---     │
│ i64   ┆ u32     │
╞═══════╪═════════╡
│ null  ┆ 4269321 │
│ 0     ┆ 62152   │
│ 1     ┆ 26818   │
└───────┴─────────┘

First few rows:
shape: (5, 21)
┌──────┬───────┬─────────┬───────┬───┬──────────┬──────────┬────────────┬─────────┐
│ YEAR ┆ HFLAG ┆ ASECWTH ┆ CPI99 ┆ … ┆ WKSWORK1 ┆ WKSWORK2 ┆ UHRSWORKLY ┆ INCWAGE │
│ ---  ┆ ---   ┆ ---     ┆ 

## 3. Data Overview

Examine the raw CPS data structure.

In [11]:
# Basic statistics
print("="*70)
print("RAW CPS DATA OVERVIEW")
print("="*70)
print(f"\nShape: {df.shape}")
print(f"\nYears: {df['YEAR'].min()} - {df['YEAR'].max()}")
print(f"Unique years: {df['YEAR'].n_unique()}")
print(f"\nObservations per year:")
print(df.group_by('YEAR').count().sort('YEAR').tail(10))
print(f"\nData types:")
print(df.schema)

RAW CPS DATA OVERVIEW

Shape: (4358291, 21)

Years: 1962 - 2022
Unique years: 61

Observations per year:
shape: (10, 2)
┌──────┬────────┐
│ YEAR ┆ count  │
│ ---  ┆ ---    │
│ i64  ┆ u32    │
╞══════╪════════╡
│ 2013 ┆ 89666  │
│ 2014 ┆ 88970  │
│ 2015 ┆ 88874  │
│ 2016 ┆ 83162  │
│ 2017 ┆ 84143  │
│ 2018 ┆ 81555  │
│ 2019 ┆ 82429  │
│ 2020 ┆ 70559  │
│ 2021 ┆ 71631  │
│ 2022 ┆ 138709 │
└──────┴────────┘

Data types:
Schema({'YEAR': Int64, 'HFLAG': Int64, 'ASECWTH': Float64, 'CPI99': Float64, 'ASECWT': Float64, 'AGE': Int64, 'SEX': Int64, 'RACE': Int64, 'EMPSTAT': Int64, 'OCC2010': String, 'OCC1990': String, 'IND1990': String, 'AHRSWORKT': Int64, 'EDUC': Int64, 'HIGRADE': Int64, 'EDUC99': String, 'CLASSWLY': Int64, 'WKSWORK1': String, 'WKSWORK2': Int64, 'UHRSWORKLY': String, 'INCWAGE': Int64})


## 4. Sample Selection

Apply sample selection filters to clean the data.

In [12]:
# Track sample size at each step
sample_flow = []

def log_sample(step, df_current):
    n = len(df_current)
    pct = (n / len(df)) * 100
    sample_flow.append({'Step': step, 'N': n, 'Percent': pct})
    print(f"{step}: {n:,} ({pct:.1f}%)")

print("="*70)
print("SAMPLE SELECTION PIPELINE")
print("="*70)
print()

# Initial
log_sample("1. Raw CPS extract", df)

# Filter 1: Valid survey weights
df = df.filter(pl.col('ASECWT').is_not_null())
log_sample("2. Valid ASECWT", df)

# Filter 2: 2014 CPS redesign adjustment
# Adjust weights for 2014 sample
df = df.with_columns([
    pl.when(pl.col('YEAR') == 2014)
    .then(pl.col('ASECWT') * (5/8 * (1 - pl.col('HFLAG')) + 3/8 * pl.col('HFLAG')))
    .otherwise(pl.col('ASECWT'))
    .alias('ASECWT')
])
print("   Applied 2014 weight adjustment")

# Filter 3: Employment type (wage/salary workers only)
# CLASSWLY: 20=private, 22=federal, 24=state, 25=local, 27=nonprofit, 28=public
kept_classes = [20, 22, 24, 25, 27, 28]
df = df.filter(pl.col('CLASSWLY').is_in(kept_classes))
log_sample("3. Wage/salary workers", df)

# Filter 4: Weeks worked
df = df.filter(pl.col('WKSWORK2') != 0)
log_sample("4. Weeks worked reported", df)

# Filter 5: Age range (16-70)
df = df.filter((pl.col('AGE') >= 16) & (pl.col('AGE') <= 70))
log_sample("5. Age 16-70", df)

print("\n" + "="*70)
print(f"Current sample: {len(df):,} observations")
print("="*70)

SAMPLE SELECTION PIPELINE

1. Raw CPS extract: 4,358,291 (100.0%)
2. Valid ASECWT: 4,219,582 (100.0%)
   Applied 2014 weight adjustment
3. Wage/salary workers: 3,658,059 (100.0%)
4. Weeks worked reported: 3,658,059 (100.0%)
5. Age 16-70: 3,658,059 (100.0%)

Current sample: 3,658,059 observations
3. Wage/salary workers: 3,658,059 (100.0%)
4. Weeks worked reported: 3,658,059 (100.0%)
5. Age 16-70: 3,658,059 (100.0%)

Current sample: 3,658,059 observations


## 5. Education Recoding

Harmonize education variables (HIGRADE pre-1992, EDUC99 post-1992) into 4 categories.

In [14]:
# Education recoding functions (robust to string/NA inputs)
def recode_higrade(val):
    '''Recode HIGRADE (pre-1992) to 4 categories'''
    try:
        if val is None:
            return None
        v = float(val)
    except Exception:
        return None
    if v < 31 or v == 999:
        return None
    elif 31 <= v < 150:
        return 'BH'  # Below High School
    elif v == 150:
        return 'HS'  # High School
    elif 150 < v < 190:
        return 'SC'  # Some College
    elif v >= 190:
        return 'CG'  # College Graduate
    return None

def recode_educ99(val):
    '''Recode EDUC99 (post-1992) to 4 categories'''
    try:
        if val is None:
            return None
        v = float(val)
    except Exception:
        return None
    if v <= 1:
        return None
    elif 1 < v < 9:
        return 'BH'
    elif v == 10:
        return 'HS'
    elif 10 < v < 15:
        return 'SC'
    elif v >= 15:
        return 'CG'
    return None

# Apply recoding (specify return dtype so Polars can infer UDF output)
print("Recoding education...")
df = df.with_columns([
    pl.col('HIGRADE').map_elements(recode_higrade, return_dtype=pl.Utf8).alias('EDUCAT_1'),
    pl.col('EDUC99').map_elements(recode_educ99, return_dtype=pl.Utf8).alias('EDUCAT_2')
])

# Combine: use HIGRADE if available, else EDUC99
df = df.with_columns([
    pl.when(pl.col('EDUCAT_1').is_not_null())
    .then(pl.col('EDUCAT_1'))
    .otherwise(pl.col('EDUCAT_2'))
    .alias('EDUCAT')
])

# Filter: education reported
df = df.filter(pl.col('EDUCAT').is_not_null())
log_sample("6. Education reported", df)

print(f"\nEducation distribution:")
print(df.group_by('EDUCAT').count().sort('EDUCAT'))

Recoding education...
6. Education reported: 3,608,182 (100.0%)

Education distribution:
shape: (4, 2)
┌────────┬─────────┐
│ EDUCAT ┆ count   │
│ ---    ┆ ---     │
│ str    ┆ u32     │
╞════════╪═════════╡
│ BH     ┆ 575634  │
│ CG     ┆ 939366  │
│ HS     ┆ 1162688 │
│ SC     ┆ 930494  │
└────────┴─────────┘
6. Education reported: 3,608,182 (100.0%)

Education distribution:
shape: (4, 2)
┌────────┬─────────┐
│ EDUCAT ┆ count   │
│ ---    ┆ ---     │
│ str    ┆ u32     │
╞════════╪═════════╡
│ BH     ┆ 575634  │
│ CG     ┆ 939366  │
│ HS     ┆ 1162688 │
│ SC     ┆ 930494  │
└────────┴─────────┘


## 6. Additional Filters - Full-Time Workers

In [16]:
# Coerce work/earnings vars to numeric and continue filtering (fixes string-vs-number ComputeError)

print("Checking WKSWORK1 and UHRSWORKLY availability (preview):")
print(availability.sort('YEAR').head(20))

# Keep years with complete data (1976+)
print("\nFiltering to years with complete data (1976+)...")
df = df.filter(pl.col('YEAR') >= 1976)
log_sample("7. Years with complete data (1976+)", df)

# Robust converter for stringy columns
def _to_float(x):
    try:
        if x is None:
            return None
        s = str(x).strip()
        if s == '' or s.upper() in {'NA', 'N/A', 'NULL', '.'}:
            return None
        return float(s)
    except Exception:
        return None

# Cast columns to floats (nullable) so numeric comparisons work
df = df.with_columns([
    pl.col('WKSWORK1').map_elements(_to_float, return_dtype=pl.Float64).alias('WKSWORK1'),
    pl.col('UHRSWORKLY').map_elements(_to_float, return_dtype=pl.Float64).alias('UHRSWORKLY'),
    pl.col('INCWAGE').map_elements(_to_float, return_dtype=pl.Float64).alias('INCWAGE'),
])

# Drop rows missing these key variables (we skip imputation for 1963-75)
df = df.filter(
    pl.col('WKSWORK1').is_not_null()
    & pl.col('UHRSWORKLY').is_not_null()
    & pl.col('INCWAGE').is_not_null()
)
log_sample("8. Has weeks/hours/wage reported", df)

# Apply weeks/hours thresholds
df = df.filter(pl.col('WKSWORK1') >= 40)
log_sample("9. >= 40 weeks worked", df)

df = df.filter(pl.col('UHRSWORKLY') >= 30)  # minimum hours threshold
log_sample("10. >= 30 hours/week", df)

# Compute HOURS_WORKED and hourly wage, guarding against zero hours
df = df.with_columns([
    (pl.col('WKSWORK1') * pl.col('UHRSWORKLY')).alias('HOURS_WORKED')
])

# Remove zero-hours to avoid division by zero
df = df.filter(pl.col('HOURS_WORKED') > 0)
log_sample("11. Positive hours worked", df)

df = df.with_columns([
    (pl.col('INCWAGE') / pl.col('HOURS_WORKED')).alias('WAGE')
])

# Wage floor: >= half minimum wage in 1999 dollars
# Min wage 1999 = $5.65/hr, so floor = $5.65 / 4 = 1.4125
df = df.filter((pl.col('WAGE') * pl.col('CPI99')) >= (5.65 / 4))
log_sample("12. Wage floor", df)

print("\n" + "="*70)
print(f"FINAL SAMPLE: {len(df):,} observations")
print("="*70)

Checking WKSWORK1 and UHRSWORKLY availability (preview):
shape: (20, 4)
┌──────┬──────────────┬────────────────┬───────┐
│ YEAR ┆ has_wkswork1 ┆ has_uhrsworkly ┆ total │
│ ---  ┆ ---          ┆ ---            ┆ ---   │
│ i64  ┆ u32          ┆ u32            ┆ u32   │
╞══════╪══════════════╪════════════════╪═══════╡
│ 1962 ┆ 0            ┆ 0              ┆ 20512 │
│ 1964 ┆ 0            ┆ 0              ┆ 21650 │
│ 1965 ┆ 0            ┆ 0              ┆ 22047 │
│ 1966 ┆ 0            ┆ 0              ┆ 47886 │
│ 1967 ┆ 0            ┆ 0              ┆ 30897 │
│ …    ┆ …            ┆ …              ┆ …     │
│ 1978 ┆ 55934        ┆ 55934          ┆ 55934 │
│ 1979 ┆ 56931        ┆ 56931          ┆ 56931 │
│ 1980 ┆ 67163        ┆ 67163          ┆ 67163 │
│ 1981 ┆ 67117        ┆ 67117          ┆ 67117 │
│ 1982 ┆ 58809        ┆ 58809          ┆ 58809 │
└──────┴──────────────┴────────────────┴───────┘

Filtering to years with complete data (1976+)...
7. Years with complete data (1976+): 3,085,97

## 7. Demographic Cell Construction

Create 264 cells: Age (11) × Race (3) × Sex (2) × Education (4)

In [17]:
# Age groups
def recode_age(age):
    if age <= 20:
        return '01'
    elif age <= 25:
        return '02'
    elif age <= 30:
        return '03'
    elif age <= 35:
        return '04'
    elif age <= 40:
        return '05'
    elif age <= 45:
        return '06'
    elif age <= 50:
        return '07'
    elif age <= 55:
        return '08'
    elif age <= 60:
        return '09'
    elif age <= 65:
        return '10'
    elif age <= 70:
        return '11'
    return None

# Race groups
def recode_race(race):
    if race == 100:
        return 'W'  # White
    elif race == 200:
        return 'B'  # Black
    else:
        return 'O'  # Other

print("Creating demographic cells...")
df = df.with_columns([
    pl.col('AGE').map_elements(recode_age).alias('AGEGROUP'),
    pl.col('RACE').map_elements(recode_race).alias('RACEGROUP')
])

# Create GROUP ID
df = df.with_columns([
    (pl.col('EDUCAT') + pl.col('AGEGROUP') + pl.col('RACEGROUP') + pl.col('SEX').cast(str)).alias('GROUP')
])

# Define skill
df = df.with_columns([
    pl.when(pl.col('EDUCAT') == 'CG')
    .then(pl.lit('S'))
    .otherwise(pl.lit('U'))
    .alias('SKILL')
])

print(f"\nCreated {df['GROUP'].n_unique()} unique demographic cells")
print(f"\nSkill distribution:")
print(df.group_by('SKILL').count())

# Show example groups
print(f"\nExample GROUP IDs:")
print(df.select(['GROUP', 'EDUCAT', 'AGEGROUP', 'RACEGROUP', 'SEX', 'SKILL']).head(10))

Creating demographic cells...

Created 264 unique demographic cells

Skill distribution:
shape: (2, 2)
┌───────┬─────────┐
│ SKILL ┆ count   │
│ ---   ┆ ---     │
│ str   ┆ u32     │
╞═══════╪═════════╡
│ U     ┆ 1696010 │
│ S     ┆ 748570  │
└───────┴─────────┘

Example GROUP IDs:
shape: (10, 6)
┌────────┬────────┬──────────┬───────────┬─────┬───────┐
│ GROUP  ┆ EDUCAT ┆ AGEGROUP ┆ RACEGROUP ┆ SEX ┆ SKILL │
│ ---    ┆ ---    ┆ ---      ┆ ---       ┆ --- ┆ ---   │
│ str    ┆ str    ┆ str      ┆ str       ┆ i64 ┆ str   │
╞════════╪════════╪══════════╪═══════════╪═════╪═══════╡
│ BH09W1 ┆ BH     ┆ 09       ┆ W         ┆ 1   ┆ U     │
│ BH10W2 ┆ BH     ┆ 10       ┆ W         ┆ 2   ┆ U     │
│ BH06W1 ┆ BH     ┆ 06       ┆ W         ┆ 1   ┆ U     │
│ BH06W2 ┆ BH     ┆ 06       ┆ W         ┆ 2   ┆ U     │
│ BH09W2 ┆ BH     ┆ 09       ┆ W         ┆ 2   ┆ U     │
│ HS05W1 ┆ HS     ┆ 05       ┆ W         ┆ 1   ┆ U     │
│ HS05W2 ┆ HS     ┆ 05       ┆ W         ┆ 2   ┆ U     │
│ BH07W1 ┆ BH     

## 8. Aggregation to Skill Groups

Aggregation steps:
1. Normalize weights within year
2. Calculate group-level averages
3. Weight by 1980 wages (efficiency units)
4. Aggregate to skilled/unskilled

In [18]:
# Step 1: Normalize weights within year
print("Step 1: Normalizing weights...")
df = df.with_columns([
    (pl.col('ASECWT') / pl.col('ASECWT').sum().over('YEAR')).alias('ASECWT_norm')
])

# Step 2: Group-level aggregation
print("Step 2: Group-level aggregation...")
grouped = df.group_by(['YEAR', 'GROUP', 'SKILL']).agg([
    (pl.col('HOURS_WORKED') * pl.col('ASECWT_norm')).sum().alias('L_group'),
    (pl.col('WAGE') * pl.col('ASECWT_norm')).sum().alias('W_group'),
    pl.col('ASECWT_norm').sum().alias('weight')
])

# Average wage per group
grouped = grouped.with_columns([
    (pl.col('W_group') / pl.col('weight')).alias('W_avg')
])

print(f"\nGrouped data shape: {grouped.shape}")
print(grouped.head())

Step 1: Normalizing weights...
Step 2: Group-level aggregation...

Grouped data shape: (11871, 7)
shape: (5, 7)
┌──────┬────────┬───────┬───────────┬──────────┬──────────┬───────────┐
│ YEAR ┆ GROUP  ┆ SKILL ┆ L_group   ┆ W_group  ┆ weight   ┆ W_avg     │
│ ---  ┆ ---    ┆ ---   ┆ ---       ┆ ---      ┆ ---      ┆ ---       │
│ i64  ┆ str    ┆ str   ┆ f64       ┆ f64      ┆ f64      ┆ f64       │
╞══════╪════════╪═══════╪═══════════╪══════════╪══════════╪═══════════╡
│ 2010 ┆ CG04B2 ┆ S     ┆ 6.28491   ┆ 0.066399 ┆ 0.002938 ┆ 22.599057 │
│ 1993 ┆ BH09B2 ┆ U     ┆ 0.583293  ┆ 0.002143 ┆ 0.000293 ┆ 7.306791  │
│ 2011 ┆ SC01W1 ┆ U     ┆ 3.716078  ┆ 0.021768 ┆ 0.00197  ┆ 11.049046 │
│ 2012 ┆ HS05W2 ┆ U     ┆ 16.849905 ┆ 0.121382 ┆ 0.008199 ┆ 14.805023 │
│ 2015 ┆ CG05O1 ┆ S     ┆ 8.137883  ┆ 0.168469 ┆ 0.003499 ┆ 48.151359 │
└──────┴────────┴───────┴───────────┴──────────┴──────────┴───────────┘

Grouped data shape: (11871, 7)
shape: (5, 7)
┌──────┬────────┬───────┬───────────┬──────────┬──

In [19]:
# Step 3: Get 1980 wages for weighting
print("Step 3: Getting 1980 wage weights...")

# Get 1980 wages by group
wages_1980 = grouped.filter(pl.col('YEAR') == 1980).select(['GROUP', 'W_avg']).rename({'W_avg': 'W_1980'})

# Join back
grouped = grouped.join(wages_1980, on='GROUP', how='left')

# Create efficiency units
grouped = grouped.with_columns([
    (pl.col('L_group') * pl.col('W_1980')).alias('L_efficiency')
])

print(f"\nGroups with 1980 wages: {grouped.filter(pl.col('W_1980').is_not_null()).shape[0]}")

# Step 4: Aggregate to skills
print("\nStep 4: Aggregating to skilled/unskilled...")
labor_series = grouped.group_by(['YEAR', 'SKILL']).agg([
    pl.col('L_efficiency').sum().alias('L'),
    (pl.col('W_avg') * pl.col('L_efficiency')).sum().alias('W_weighted')
])

# Calculate average wages
labor_series = labor_series.with_columns([
    (pl.col('W_weighted') / pl.col('L')).alias('W')
])

print(f"\nAggregated shape: {labor_series.shape}")
print(labor_series.head(10))

Step 3: Getting 1980 wage weights...

Groups with 1980 wages: 11513

Step 4: Aggregating to skilled/unskilled...

Aggregated shape: (92, 5)
shape: (10, 5)
┌──────┬───────┬──────────────┬───────────────┬───────────┐
│ YEAR ┆ SKILL ┆ L            ┆ W_weighted    ┆ W         │
│ ---  ┆ ---   ┆ ---          ┆ ---           ┆ ---       │
│ i64  ┆ str   ┆ f64          ┆ f64           ┆ f64       │
╞══════╪═══════╪══════════════╪═══════════════╪═══════════╡
│ 1992 ┆ S     ┆ 5498.849678  ┆ 100365.81522  ┆ 18.252147 │
│ 2000 ┆ U     ┆ 10207.449805 ┆ 151590.138364 ┆ 14.850932 │
│ 1999 ┆ S     ┆ 6248.425895  ┆ 158109.681696 ┆ 25.303922 │
│ 1986 ┆ U     ┆ 10383.511659 ┆ 100577.412923 ┆ 9.686262  │
│ 1981 ┆ U     ┆ 10681.963668 ┆ 80870.325718  ┆ 7.570736  │
│ 2007 ┆ S     ┆ 6907.211659  ┆ 232277.614291 ┆ 33.628275 │
│ 2021 ┆ U     ┆ 7915.117845  ┆ 196482.7332   ┆ 24.823728 │
│ 2008 ┆ S     ┆ 7091.396314  ┆ 236752.045725 ┆ 33.385815 │
│ 1990 ┆ S     ┆ 5376.742807  ┆ 92124.205951  ┆ 17.133832 │
│ 201

## 9. Create Final Output

Pivot to wide format and save.

In [20]:
# Pivot to wide format
print("Creating final output format...")

# Separate skilled and unskilled
labor_wide = labor_series.pivot(
    values=['L', 'W'],
    index='YEAR',
    columns='SKILL'
).sort('YEAR')

# Rename columns
labor_wide = labor_wide.rename({
    'L_S': 'L_S',
    'L_U': 'L_U',
    'W_S': 'W_S', 
    'W_U': 'W_U'
})

# Calculate ratios
labor_wide = labor_wide.with_columns([
    (pl.col('W_S') / pl.col('W_U')).alias('SKILL_PREMIUM'),
    (pl.col('L_S') / pl.col('L_U')).alias('LABOR_INPUT_RATIO')
])

print(f"\nFinal output shape: {labor_wide.shape}")
print(f"\nColumns: {labor_wide.columns}")
print(f"\nFirst few years:")
print(labor_wide.head(10))

# Save
output_file = DATA_PROC / 'labor_totl_python.csv'
labor_wide.write_csv(output_file)
print(f"\n✅ Saved to: {output_file}")

Creating final output format...

Final output shape: (46, 7)

Columns: ['YEAR', 'L_S', 'L_U', 'W_S', 'W_U', 'SKILL_PREMIUM', 'LABOR_INPUT_RATIO']

First few years:
shape: (10, 7)
┌──────┬─────────────┬──────────────┬───────────┬──────────┬───────────────┬───────────────────┐
│ YEAR ┆ L_S         ┆ L_U          ┆ W_S       ┆ W_U      ┆ SKILL_PREMIUM ┆ LABOR_INPUT_RATIO │
│ ---  ┆ ---         ┆ ---          ┆ ---       ┆ ---      ┆ ---           ┆ ---               │
│ i64  ┆ f64         ┆ f64          ┆ f64       ┆ f64      ┆ f64           ┆ f64               │
╞══════╪═════════════╪══════════════╪═══════════╪══════════╪═══════════════╪═══════════════════╡
│ 1976 ┆ 4056.453573 ┆ 11183.259719 ┆ 7.590603  ┆ 5.212479 ┆ 1.456237      ┆ 0.362726          │
│ 1977 ┆ 4089.863988 ┆ 11113.543068 ┆ 8.062356  ┆ 5.563399 ┆ 1.449178      ┆ 0.368007          │
│ 1978 ┆ 4103.251214 ┆ 11130.908084 ┆ 8.605731  ┆ 5.947687 ┆ 1.446904      ┆ 0.368636          │
│ 1979 ┆ 4184.476323 ┆ 11023.536982 ┆ 9.07499

## 10. Summary

### What We Did

✅ Loaded raw CPS (4.4M observations)  
✅ Applied all sample selection filters  
✅ Recoded education variables  
✅ Created 264 demographic cells  
✅ Aggregated using 1980 wage weights  
✅ Produced skilled/unskilled series  

### Output

**File**: `labor_totl_python.csv`

Contains:
- `YEAR`: 1976-2018 (1963-75 needs imputation)
- `L_S`, `L_U`: Labor input (efficiency units)
- `W_S`, `W_U`: Average wages (1999 dollars)
- `SKILL_PREMIUM`: W_S / W_U
- `LABOR_INPUT_RATIO`: L_S / L_U

### Next Steps

1. Implement 1963-75 imputation for full time series
2. Add industry segmentation
3. Run analysis notebooks to visualize and analyze output

---

**Processing complete!**

## Generate Labor Share Table for Manuscript

Create Table~\ref{tab:labor_share_by_industry} showing initial level, final level, change, and growth rate.

In [21]:
import pandas as pd
import numpy as np
from pathlib import Path

# Paths
ROOT = Path.cwd().parent
DATA_IND = ROOT / 'data' / 'proc' / 'ind'
CROSSWALK = ROOT / 'data' / 'cross_walk.csv'

# Load crosswalk for industry names
crosswalk = pd.read_csv(CROSSWALK)
code_to_name = dict(zip(crosswalk['code_bea'].str.upper(), crosswalk['ind_desc']))

print(f"Found {len(crosswalk)} industries in crosswalk")
print(f"\nSample mappings:")
for code, name in list(code_to_name.items())[:5]:
    print(f"  {code}: {name}")

# Process each industry file
results = []

for file in sorted(DATA_IND.glob('*.csv')):
    try:
        # Get industry code from filename
        ind_code = file.stem.upper()
        
        # Load data
        df = pd.read_csv(file)
        
        # Check if L_SHARE column exists
        if 'L_SHARE' not in df.columns:
            print(f"Warning: {file.name} missing L_SHARE column")
            continue
        
        # Get initial and final years
        df = df.sort_values('YEAR')
        initial_year = df['YEAR'].min()
        final_year = df['YEAR'].max()
        
        # Get labor share values
        initial_ls = df.loc[df['YEAR'] == initial_year, 'L_SHARE'].values[0]
        final_ls = df.loc[df['YEAR'] == final_year, 'L_SHARE'].values[0]
        
        # Calculate change and annualized growth rate
        change = final_ls - initial_ls
        years = final_year - initial_year
        if years > 0 and initial_ls > 0:
            annual_growth = ((final_ls / initial_ls) ** (1/years) - 1) * 100
        else:
            annual_growth = np.nan
        
        # Get industry name from crosswalk
        industry_name = code_to_name.get(ind_code, ind_code)
        
        results.append({
            'Industry': industry_name,
            'Code': ind_code,
            'Initial Year': int(initial_year),
            'Initial LS': initial_ls,
            'Final Year': int(final_year),
            'Final LS': final_ls,
            'Change': change,
            'Annual Growth (%)': annual_growth
        })
        
    except Exception as e:
        print(f"Error processing {file.name}: {e}")

# Create DataFrame
labor_share_table = pd.DataFrame(results)

# Sort by initial labor share (descending)
labor_share_table = labor_share_table.sort_values('Initial LS', ascending=False)

print(f"\n✅ Processed {len(labor_share_table)} industries")
print(f"\nFirst 10 industries by initial labor share:")
print(labor_share_table.head(10))

Found 59 industries in crosswalk

Sample mappings:
  110C: Farms
  113F: Forestry, fishing, and related activities
  2110: Oil and gas extraction
  2120: Mining, except oil and gas
  2130: Support activities for mining

✅ Processed 56 industries

First 10 industries by initial labor share:
                                        Industry   Code  Initial Year  \
52                                         711AS  711AS          1987   
44  Computer systems design and related services   5415          1987   
49                                           621    621          1987   
51                                           624    624          1987   
6                                             23     23          1987   
42                                Legal services   5411          1987   
48                                            61     61          1987   
50                                         622HO  622HO          1987   
12                                           323    

In [22]:
# Display full table
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', '{:.4f}'.format)

print("="*100)
print("LABOR SHARE STATISTICS BY INDUSTRY")
print("="*100)
print(f"\nCoverage: {labor_share_table['Initial Year'].min()}-{labor_share_table['Final Year'].max()}")
print(f"Number of industries: {len(labor_share_table)}")
print(f"\nIndustries with DECLINING labor share: {(labor_share_table['Change'] < 0).sum()} ({(labor_share_table['Change'] < 0).sum() / len(labor_share_table) * 100:.1f}%)")
print(f"Industries with INCREASING labor share: {(labor_share_table['Change'] > 0).sum()} ({(labor_share_table['Change'] > 0).sum() / len(labor_share_table) * 100:.1f}%)")
print(f"\nMedian change: {labor_share_table['Change'].median():.4f}")
print(f"Mean annual growth: {labor_share_table['Annual Growth (%)'].mean():.2f}%")
print(f"\n{labor_share_table.to_string(index=False)}")

LABOR SHARE STATISTICS BY INDUSTRY

Coverage: 1987-2018
Number of industries: 56

Industries with DECLINING labor share: 46 (82.1%)
Industries with INCREASING labor share: 10 (17.9%)

Median change: -0.0691
Mean annual growth: -0.47%

                                    Industry   Code  Initial Year  Initial LS  Final Year  Final LS  Change  Annual Growth (%)
                                       711AS  711AS          1987      0.9711        2018    0.7670 -0.2041            -0.7582
Computer systems design and related services   5415          1987      0.9632        2018    0.8607 -0.1024            -0.3621
                                         621    621          1987      0.9361        2018    0.8677 -0.0684            -0.2443
                                         624    624          1987      0.9326        2018    0.9496  0.0170             0.0582
                                          23     23          1987      0.9240        2018    0.7979 -0.1261            -0.4723
   

In [23]:
# Generate LaTeX table for manuscript
# Simplify for publication - group similar industries and show top/bottom performers

def generate_latex_table(df, max_rows=20):
    """Generate LaTeX table code for manuscript"""
    
    # Select top declining and top stable/increasing
    df_sorted = df.sort_values('Change')
    top_declining = df_sorted.head(10)
    top_stable = df_sorted.tail(10)
    
    # Combine
    display_df = pd.concat([top_declining, top_stable])
    
    latex = r"""\begin{table}[H]
\centering
\caption{Labor Share Statistics by Industry}
\label{tab:labor_share_by_industry}
\scriptsize
\begin{tabularx}{\textwidth}{Xcccc}
\toprule
Industry & Initial LS & Final LS & Change & Growth (\%/yr) \\
\midrule
"""
    
    # Add largest declines section
    latex += r"\multicolumn{5}{l}{\textit{Largest Declines}} \\" + "\n"
    for _, row in top_declining.iterrows():
        latex += f"{row['Industry']} & {row['Initial LS']:.3f} & {row['Final LS']:.3f} & {row['Change']:.3f} & {row['Annual Growth (%)']:.2f} \\\\\n"
    
    latex += r"\midrule" + "\n"
    latex += r"\multicolumn{5}{l}{\textit{Most Stable / Increasing}} \\" + "\n"
    
    # Add most stable/increasing section
    for _, row in top_stable.iterrows():
        latex += f"{row['Industry']} & {row['Initial LS']:.3f} & {row['Final LS']:.3f} & {row['Change']:.3f} & {row['Annual Growth (%)']:.2f} \\\\\n"
    
    latex += r"""\bottomrule
\end{tabularx}
\begin{minipage}{\textwidth}
\vspace{0.2cm}
\footnotesize
\textit{Notes:} Initial LS is labor share in initial year (typically 1987), Final LS is labor share in final year (typically 2018). 
Change is percentage point change. Growth is annualized percentage growth rate. 
Table shows 10 industries with largest declines and 10 most stable/increasing industries out of """ + str(len(df)) + r""" total industries.
Labor share declining in """ + str((df['Change'] < 0).sum()) + r""" industries (""" + f"{(df['Change'] < 0).sum() / len(df) * 100:.0f}" + r"""\%).
\end{minipage}
\end{table}"""
    
    return latex

# Generate and save LaTeX
latex_code = generate_latex_table(labor_share_table)

# Save to file
output_tex = ROOT / 'documents' / 'tables' / 'labor_share_by_industry.tex'
output_tex.parent.mkdir(exist_ok=True, parents=True)
with open(output_tex, 'w') as f:
    f.write(latex_code)

print("="*100)
print("LATEX TABLE GENERATED")
print("="*100)
print(f"\n✅ Saved to: {output_tex}")
print(f"\nPreview:\n")
print(latex_code[:1000] + "\n...\n")

LATEX TABLE GENERATED

✅ Saved to: /Users/mitchv34/Work/industry_skill_premium/documents/tables/labor_share_by_industry.tex

Preview:

\begin{table}[H]
\centering
\caption{Labor Share Statistics by Industry}
\label{tab:labor_share_by_industry}
\scriptsize
\begin{tabularx}{\textwidth}{Xcccc}
\toprule
Industry & Initial LS & Final LS & Change & Growth (\%/yr) \\
\midrule
\multicolumn{5}{l}{\textit{Largest Declines}} \\
524 & 0.834 & 0.508 & -0.326 & -1.59 \\
212 & 0.642 & 0.325 & -0.316 & -2.17 \\
482 & 0.825 & 0.531 & -0.294 & -1.41 \\
324 & 0.390 & 0.107 & -0.283 & -4.08 \\
331 & 0.790 & 0.508 & -0.282 & -1.41 \\
Legal services & 0.918 & 0.664 & -0.254 & -1.04 \\
323 & 0.877 & 0.665 & -0.212 & -0.89 \\
711AS & 0.971 & 0.767 & -0.204 & -0.76 \\
327 & 0.693 & 0.500 & -0.193 & -1.05 \\
334 & 0.711 & 0.521 & -0.191 & -1.00 \\
\midrule
\multicolumn{5}{l}{\textit{Most Stable / Increasing}} \\
624 & 0.933 & 0.950 & 0.017 & 0.06 \\
22 & 0.247 & 0.265 & 0.018 & 0.23 \\
493 & 0.824 & 0.870 & 0.0

In [24]:
# Also save CSV for reference
output_csv = ROOT / 'data' / 'results' / 'labor_share_by_industry.csv'
output_csv.parent.mkdir(exist_ok=True, parents=True)
labor_share_table.to_csv(output_csv, index=False)

print(f"✅ CSV saved to: {output_csv}")
print(f"\n{'='*100}")
print("SUMMARY STATISTICS")
print('='*100)
print(f"\nTotal industries: {len(labor_share_table)}")
print(f"Declining labor share: {(labor_share_table['Change'] < 0).sum()} ({(labor_share_table['Change'] < 0).sum() / len(labor_share_table) * 100:.1f}%)")
print(f"Increasing labor share: {(labor_share_table['Change'] > 0).sum()} ({(labor_share_table['Change'] > 0).sum() / len(labor_share_table) * 100:.1f}%)")
print(f"\nLabor Share Statistics:")
print(f"  Mean initial: {labor_share_table['Initial LS'].mean():.4f}")
print(f"  Mean final: {labor_share_table['Final LS'].mean():.4f}")
print(f"  Mean change: {labor_share_table['Change'].mean():.4f}")
print(f"  Median change: {labor_share_table['Change'].median():.4f}")
print(f"  Mean annual growth: {labor_share_table['Annual Growth (%)'].mean():.3f}%")
print(f"  Median annual growth: {labor_share_table['Annual Growth (%)'].median():.3f}%")
print(f"\nLargest decline: {labor_share_table.loc[labor_share_table['Change'].idxmin(), 'Industry']}")
print(f"  Change: {labor_share_table['Change'].min():.4f}")
print(f"\nLargest increase: {labor_share_table.loc[labor_share_table['Change'].idxmax(), 'Industry']}")
print(f"  Change: {labor_share_table['Change'].max():.4f}")

✅ CSV saved to: /Users/mitchv34/Work/industry_skill_premium/data/results/labor_share_by_industry.csv

SUMMARY STATISTICS

Total industries: 56
Declining labor share: 46 (82.1%)
Increasing labor share: 10 (17.9%)

Labor Share Statistics:
  Mean initial: 0.6713
  Mean final: 0.5906
  Mean change: -0.0806
  Median change: -0.0691
  Mean annual growth: -0.472%
  Median annual growth: -0.398%

Largest decline: 524
  Change: -0.3263

Largest increase: 113FF
  Change: 0.2048


## Generate Manuscript Tables

To generate all tables and figures for the manuscript Data Description section, run:

```bash
python scripts/data_processing/generate_manuscript_tables.py
```

This script uses the labor share data generated above and creates:
- Aggregate summary statistics by decade
- Industry-level trend correlations  
- Labor share heterogeneity groups
- Slope distribution figure

All outputs are saved to `documents/tables/` and `documents/images/`.